In [1]:
import numpy as np
import nibabel as nib
import os
from nilearn import plotting
from nilearn import image
from nilearn import input_data
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline